In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [11]:
# Import data
df = pd.read_csv('../../../../docs/data/cycpeptdb_clean.csv', header=0)
filtered_df = df[df['Permeability'] != -10]

# Select the desired columns
columns = ['TPSA',  # topological polar surface area
           'MolWt',  # molecular weight
           'NumHAcceptors',  # number of hydrogen bond acceptors
           'NumHDonors',  # number of hydrogen bond donors
           'NumHAcceptors'] + [col for col in filtered_df.columns if col.startswith('fr_')]

# Create the feature matrix and target vector
X = filtered_df[columns].values
X = np.hstack((X, np.ones((X.shape[0], 1))))  # Add a column of ones for the bias term
y = filtered_df['Permeability'].values

# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Random Forest Regressor': RandomForestRegressor(random_state=42),
    'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42)
}

# Initialize a dictionary to store results
results = {}

# Train and evaluate each model
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on validation set
    y_val_pred = model.predict(X_val)
    
    # Calculate metrics
    mse = mean_squared_error(y_val, y_val_pred)
    mae = mean_absolute_error(y_val, y_val_pred)
    r2 = r2_score(y_val, y_val_pred)
    
    # Store the results
    results[name] = {"MSE": mse, "MAE": mae, "R2": r2}

# Print the results
for name, metrics in results.items():
    print(f"{name}:")
    print(f"  MSE: {metrics['MSE']}")
    print(f"  MAE: {metrics['MAE']}")
    print(f"  R2: {metrics['R2']}")
    print()

# Evaluate the best model on the test set
# Here, we assume Random Forest Regressor is the best model based on validation performance
best_model = models["Random Forest Regressor"]
y_test_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Test set evaluation (Random Forest Regressor):")
print(f"  MSE: {test_mse}")
print(f"  MAE: {test_mae}")
print(f"  R2: {test_r2}")


Linear Regression:
  MSE: 0.4388729722076431
  MAE: 0.5013255445526819
  R2: 0.23752747213154435

Ridge Regression:
  MSE: 0.4409064401180686
  MAE: 0.5028828539762142
  R2: 0.23399464255171787

Lasso Regression:
  MSE: 0.5759167290588226
  MAE: 0.5806486519634247
  R2: -0.0005644276482186239

Random Forest Regressor:
  MSE: 0.2434612377870303
  MAE: 0.35779934117653134
  R2: 0.5770245215154597

Gradient Boosting Regressor:
  MSE: 0.3025066713312534
  MAE: 0.4154160126061942
  R2: 0.4744423990687573

Test set evaluation (Random Forest Regressor):
  MSE: 0.24083278530129623
  MAE: 0.3510326724301126
  R2: 0.5890653934520796


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Import data
df = pd.read_csv('../../../../docs/data/cycpeptdb_clean.csv', header=0)
filtered_df = df[df['Permeability'] != -10]
filtered_df = filtered_df.drop('SMILES', axis=1)

# Select the desired columns
columns = ['TPSA',
           'MolWt',
           'NumHAcceptors', 
           'NumHDonors',
           'NumRotatableBonds',
           'MaxPartialCharge',
           'MinPartialCharge',
           'NHOHCount',
           'NOCount',
           'NumHeteroatoms',
           'NumRotatableBonds',
           'NumSaturatedCarbocycles',
           'NumSaturatedHeterocycles',
           'NumSaturatedRings',
           'RingCount',] + [col for col in filtered_df.columns if col.startswith('fr_')]

# Create the feature matrix and target vector
X = filtered_df[columns].values
X = np.hstack((X, np.ones((X.shape[0], 1))))  # Add a column of ones for the bias term
y = filtered_df['Permeability'].values

# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Random Forest Regressor': RandomForestRegressor(random_state=42),
    'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42)
}

# Initialize a dictionary to store results
results = {}
feature_weights = {col: [] for col in columns + ['Bias']}  # Initialize a dictionary for feature weights

# Train and evaluate each model
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on validation set
    y_val_pred = model.predict(X_val)
    
    # Calculate metrics
    mse = mean_squared_error(y_val, y_val_pred)
    mae = mean_absolute_error(y_val, y_val_pred)
    r2 = r2_score(y_val, y_val_pred)
    
    # Store the results
    results[name] = {"MSE": mse, "MAE": mae, "R2": r2}
    
    # Extract feature weights or importances
    if hasattr(model, 'coef_'):
        weights = model.coef_
    elif hasattr(model, 'feature_importances_'):
        weights = model.feature_importances_
    else:
        weights = np.zeros(len(columns) + 1)  # Default to zero if the model doesn't support feature importances

    # Append the feature weights to the dictionary
    for i, col in enumerate(columns + ['Bias']):
        feature_weights[col].append(weights[i])

# Create a DataFrame for feature weights
feature_weights_df = pd.DataFrame(feature_weights, index=models.keys()).T

# Sort the DataFrame by the absolute sum of weights for each feature across all models
feature_weights_df['SumAbsWeights'] = feature_weights_df.abs().sum(axis=1)
feature_weights_df = feature_weights_df.sort_values(by='SumAbsWeights', ascending=False)
feature_weights_df = feature_weights_df.drop(columns='SumAbsWeights')

# Print the feature weights DataFrame
print(feature_weights_df)

# Print the results
for name, metrics in results.items():
    print(f"{name}:")
    print(f"  MSE: {metrics['MSE']}")
    print(f"  MAE: {metrics['MAE']}")
    print(f"  R2: {metrics['R2']}")
    print()

# Optionally, you can also evaluate the best model on the test set
# Here, we assume Gradient Boosting Regressor is the best model based on validation performance
best_model = models["Random Forest Regressor"]
y_test_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Test set evaluation (Random Forest Regressor):")
print(f"  MSE: {test_mse}")
print(f"  MAE: {test_mae}")
print(f"  R2: {test_r2}")


              Linear Regression  Ridge Regression  Lasso Regression  \
fr_guanido            -4.329978         -0.282532              -0.0   
fr_priamide           -2.012878         -1.729224              -0.0   
fr_thiophene           2.506233          0.658698               0.0   
fr_HOCCN              -1.703406         -1.334704              -0.0   
fr_amide              -2.431845         -0.012600               0.0   
...                         ...               ...               ...   
fr_isocyan             0.000000          0.000000               0.0   
fr_hdrzone             0.000000          0.000000               0.0   
fr_hdrzine             0.000000          0.000000               0.0   
fr_furan               0.000000          0.000000               0.0   
Bias                   0.000000          0.000000               0.0   

              Random Forest Regressor  Gradient Boosting Regressor  
fr_guanido                   0.000032                     0.000060  
fr_priami

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Import data
df = pd.read_csv('../../../../docs/data/cycpeptdb_clean.csv', header=0)
filtered_df = df[df['Permeability'] != -10]
filtered_df = filtered_df.drop('SMILES', axis=1)

# Select the desired columns
columns = ['TPSA', 'MolWt', 'NumHAcceptors', 'NumHDonors', 'NumHAcceptors'] + [col for col in filtered_df.columns if col.startswith('fr_')]

# Create the feature matrix and target vector
X = filtered_df[columns].values
X = np.hstack((X, np.ones((X.shape[0], 1))))  # Add a column of ones for the bias term
y = filtered_df['Permeability'].values

# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Random Forest Regressor': RandomForestRegressor(random_state=42),
    'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42)
}

# Initialize a dictionary to store results
results = {}
feature_importances = {}

# Train and evaluate each model
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on validation set
    y_val_pred = model.predict(X_val)
    
    # Calculate metrics
    mse = mean_squared_error(y_val, y_val_pred)
    mae = mean_absolute_error(y_val, y_val_pred)
    r2 = r2_score(y_val, y_val_pred)
    
    # Store the results
    results[name] = {"MSE": mse, "MAE": mae, "R2": r2}
    
    # Store feature importances or coefficients
    if hasattr(model, 'coef_'):
        feature_importances[name] = model.coef_
    elif hasattr(model, 'feature_importances_'):
        feature_importances[name] = model.feature_importances_

# Print the results
for name, metrics in results.items():
    print(f"{name}:")
    print(f"  MSE: {metrics['MSE']}")
    print(f"  MAE: {metrics['MAE']}")
    print(f"  R2: {metrics['R2']}")
    print()

# Print feature importances
for name, importances in feature_importances.items():
    print(f"{name} feature importances:")
    for feature, importance in zip(columns, importances):
        print(f"  {feature}: {importance}")
    print()

# Optionally, you can also evaluate the best model on the test set
# Here, we assume Gradient Boosting Regressor is the best model based on validation performance
best_model = models["Random Forest Regressor"]
y_test_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Test set evaluation (Random Forest Regressor):")
print(f"  MSE: {test_mse}")
print(f"  MAE: {test_mae}")
print(f"  R2: {test_r2}")



Linear Regression:
  MSE: 0.4386944823603446
  MAE: 0.5015397236820028
  R2: 0.23783757007260453

Ridge Regression:
  MSE: 0.4391635922024334
  MAE: 0.5029112788421397
  R2: 0.23702256575519076

Lasso Regression:
  MSE: 0.5578388522980439
  MAE: 0.5747915821761139
  R2: 0.03084303023862489

Random Forest Regressor:
  MSE: 0.24331053675875677
  MAE: 0.3576866603522185
  R2: 0.5772863407689948

Gradient Boosting Regressor:
  MSE: 0.3030205942562033
  MAE: 0.4155815820428958
  R2: 0.4735495390921107

Linear Regression feature importances:
  TPSA: 0.07581458441096238
  MolWt: 0.00047569759001326456
  NumHAcceptors: -0.8403284551508294
  NumHDonors: -0.019689233180116394
  NumHAcceptors: -0.8403284551507035
  fr_Al_COO: -0.7370548163822588
  fr_Al_OH: -0.18747274465399355
  fr_Al_OH_noTert: 0.6641055228708497
  fr_ArN: -2.6645352591003757e-15
  fr_Ar_COO: -1.1102230246251565e-14
  fr_Ar_N: -0.32469889545598163
  fr_Ar_NH: -0.7098686561235745
  fr_Ar_OH: -0.18030092619841298
  fr_COO: -0.737

In [ ]:
ghghgk